# Patchseq Procedures: V1 to V2 Conversion with Specimen Procedures

This notebook demonstrates the core workflow for processing patchseq metadata:
1. Load V1 procedures for a mouse from the metadata service
2. Convert V1 to V2 schema format
3. Load specimen procedures from Excel file
4. Add specimen procedures to the V2 record
5. Save the final procedures.json file

In [16]:
# Import required modules
from metadata_service import fetch_procedures_metadata
from schema_conversion import convert_procedures_to_v2
from excel_loader import load_specimen_procedures_excel, get_specimen_procedures_for_subject
from file_io import save_procedures_json

print("Modules imported successfully!")

Modules imported successfully!


In [17]:
# Define the mouse ID to process and Excel file path
mouse_id = "699461"  # Example mouse from your dataset
excel_file = "DT_HM_TissueClearingTracking_.xlsx"

print(f"Processing mouse: {mouse_id}")
print(f"Excel file: {excel_file}")

Processing mouse: 699461
Excel file: DT_HM_TissueClearingTracking_.xlsx


In [18]:
# Step 1: Load V1 procedures and convert to V2
success, v1_data, message = fetch_procedures_metadata(mouse_id)
print(f"API Success: {success}, Message: {message}")
print(f"V1 data keys: {list(v1_data.keys()) if v1_data else 'None'}")

v2_data = convert_procedures_to_v2(v1_data)[0]


  Fetching procedures from: http://aind-metadata-service/procedures/699461
    Response status: 406
API Success: True, Message: Success (with validation warnings): Validation Errors: 11 validation errors for Procedures
subject_procedures.0.Surgery.protocol_id
  Field required [type=missing, input_value={'procedure_type': 'Surge...1b/v6'}], 'notes': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
subject_procedures.1.Surgery.procedures.0.Headframe.protocol_id
  Field required [type=missing, input_value={'procedure_type': 'Headf... 'well_type': 'No Well'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
subject_procedures.1.Surgery.procedures.0.Headframe.headframe_part_number
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
subject_procedures.1.Surgery.procedures.1.Nano

In [24]:
# Step 2: Load specimen procedures from Excel and add to V2 data
excel_data = load_specimen_procedures_excel(excel_file)
specimen_procedures = get_specimen_procedures_for_subject(mouse_id, excel_data)
v2_data['specimen_procedures'] = []
v2_data['specimen_procedures'].extend(specimen_procedures)

print(f"Added {len(specimen_procedures)} specimen procedures")
# print(f"Total specimen procedures: {len(v2_data['specimen_procedures'])}")

Loading Excel file: DT_HM_TissueClearingTracking_.xlsx
Found 14 total sheets

--- Loading 'Batch Info' sheet ---
✓ Batch Info loaded - Shape: (147, 5)
  Columns: ['Batch #', 'LabTrack IDs', 'Notes', 'Total #', 'Column_4']

--- Loading 'Dec 2022 - Feb 2023' sheet ---
✓ Dec 2022 - Feb 2023 loaded - Shape: (34, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Mar - May 2023' sheet ---
✓ Mar - May 2023 loaded - Shape: (45, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Jun - Aug 2023' sheet ---
✓ Jun - Aug 2023 loaded - Shape: (54, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Sep - Nov 2023' sheet ---
✓ Sep - Nov 2023 loaded - Shape: (32, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'In

In [25]:
v2_data['specimen_procedures']

[[SpecimenProcedure(object_type='Specimen procedure', procedure_type='Fixation', procedure_name='SHIELD OFF', specimen_id='699461', start_date=datetime.date(2024, 3, 29), end_date=datetime.date(2024, 4, 1), experimenters=['Holly Myers'], protocol_id=None, protocol_parameters=None, procedure_details=[Reagent(object_type='Reagent', name='SHIELD Buffer', source=_Lifecanvas(name='LifeCanvas', abbreviation=None, registry=None, registry_identifier=None), rrid=None, lot_number='bu19', expiration_date=None), Reagent(object_type='Reagent', name='SHIELD Epoxy', source=_Lifecanvas(name='LifeCanvas', abbreviation=None, registry=None, registry_identifier=None), rrid=None, lot_number='ex43', expiration_date=None)], notes='stored in 1xpbs in 4c 4/2/24 - 4/3/24'),
  SpecimenProcedure(object_type='Specimen procedure', procedure_type='Fixation', procedure_name='SHIELD ON', specimen_id='699461', start_date=datetime.date(2024, 4, 1), end_date=datetime.date(2024, 4, 2), experimenters=['Holly Myers'], proto

In [27]:
# Step 3: Save the final procedures.json file
save_procedures_json(mouse_id, v2_data, is_v1=False)

print(f"Saved procedures.json for mouse {mouse_id}")
print(f"Final: {len(v2_data)} procedures, {len(v2_data['specimen_procedures'])} specimen procedures")

    Saved procedures.json to: procedures/699461/procedures.json
Saved procedures.json for mouse 699461
Final: 3 procedures, 2 specimen procedures
